<a href="https://colab.research.google.com/github/Jihyun13579/ESAA/blob/main/OBProject1_Lig.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# matplotlib 한글 깨짐방지
# 아래코드 실행후 '런타임 다시시작' 후 모듈 불러오기

!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

In [3]:
np.set_printoptions(suppress = True)

In [4]:
plt.rc('font', family='NanumBarunGothic') # matplotlib 한글 깨짐 방지

## 1. 데이터 불러오기

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA 학회_OB/OB프로젝트1/train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/ESAA 학회_OB/OB프로젝트1/test.csv')

In [7]:
train.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0.472227,-1.756525,0.734336,9.126398,1,0,1,0,0,1,0,0
1,0.543313,-1.033082,-0.911192,9.052009,1,0,1,0,1,0,0,0
2,0.898745,-0.943687,-0.911192,10.207990,1,0,1,0,0,0,1,0
3,-0.025379,0.622393,3.202629,9.059265,1,0,1,0,0,0,1,0
4,1.040918,-1.504893,1.557100,9.431590,1,0,1,0,0,1,0,0


In [8]:
test.head()

,age,bmi,children,charges,sex_female,sex_male,smoker_no,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,0.401140,-0.891539,0.734336,9095.06825,1,0,1,0,1,0,0,0
1,-0.238638,-0.089461,-0.911192,5272.17580,1,0,1,0,0,1,0,0
2,1.751782,-0.608453,-0.911192,29330.98315,1,0,0,1,0,1,0,0
3,0.472227,-0.797177,1.557100,9301.89355,0,1,1,0,0,1,0,0
4,-1.447107,0.225079,-0.911192,33750.29180,0,1,0,1,0,1,0,0


## 2. 2. Feature, Target 분리

In [9]:
X_train = train.drop(['charges'], axis = 1)
y_train = train['charges']

X_test = test.drop(['charges'], axis = 1)
y_test = test['charges']

## 3. 평가 지표

#### 현재, y가 로그 변환되어있는 데이터 >  test 데이터로 예측 후 예측 성능을 평가할 때는 원래 스케일로 복원해주는 과정이 필요합니다!

- 즉, 실제 값, 예측값 모두 실제의 로그 변환 값 > 예측 결과 오류에 RMSE만 적용 -> RMSLE 자동 측정됨

- get_mse(model) : 단일 모델의 RMSE 값 반환
- get_rmses(models) : get_rmse()를 이용해 여러 모델의 RMSE 값 반환

In [10]:
from sklearn.metrics import mean_squared_error

In [11]:
def get_rmse(model):
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    rmse = np.sqrt(mse)
    print(model.__class__.__name__, '로그 변환된 RMSE:', np.round(rmse,3))
    return rmse

def get_rmses(models):
    rmses=[]
    for model in models:
        rmse=get_rmse(model)
        rmses.append(rmse)
    return rmses

## 4. LightGBM 학습시키기

In [12]:
# 영향을 끼치는 회귀계수 추출하기
def get_top_bottom_coef(model, n=10):
    # coef_ 속성을 기반으로 Series 객체를 생성. index는 칼럼명.
    coef = pd.Series(model.coef_, index=X_features.columns)

    # + 상위 10개, - 하위 10개의 회귀 계수를 추출해 반환.
    coef_high = coef.sort_values(ascending=False).head(n)
    coef_low = coef.sort_values(ascending=False).tail(n)
    return coef_high, coef_low

In [13]:
# 회귀 계수를 시각화하는 함수

def plot_coef(columns, coef):
  coef_df = pd.DataFrame(list(zip(columns, coef)))
  coef_df.columns=['feature', 'coef']
  coef_df = coef_df.sort_values('coef', ascending=False).reset_index(drop=True)

  fig, ax = plt.subplots(figsize=(9, 7))
  ax.barh(np.arange(len(coef_df)), coef_df['coef'])
  idx = np.arange(len(coef_df))
  ax.set_yticks(idx)
  ax.set_yticklabels(coef_df['feature'])
  fig.tight_layout()
  plt.show()

In [14]:
# 교차검증을 위한 코드
from sklearn.model_selection import cross_val_score

def get_avg_rmse_cv(models):

    for model in models:
        # 분할하지 않고 전체 데이터로 cross_val_score() 수행. 모델별 CV RMSE값과 평균 RMSE 출력
        rmse_list = np.sqrt(-cross_val_score(model, X_features, y_target,
                                            scoring='neg_mean_squared_error', cv=5))
        rmse_avg = np.mean(rmse_list)
        print('\n{0} CV RMSE 값 리스트: {1}'.format(model.__class__.__name__, np.round(rmse_list,3)))
        print('{0} CV 평균 RMSE 값: {1}'.format(model.__class__.__name__, np.round(rmse_avg, 3)))

In [15]:
# gridsearch 시켜 최적의 하이퍼 파라미터 구하는 함수
from sklearn.model_selection import GridSearchCV

def print_best_params(model, params):
    grid_model = GridSearchCV(model, param_grid = params,
                             scoring='neg_mean_squared_error', cv=5)
    grid_model.fit(X_train, y_train)
    rmse = np.sqrt(-1*grid_model.best_score_)
    print('{0} 5 CV 시 최적 평균 RMSE 값:{1}, 최적 alpha:{2}'.format(model.__class__.__name__,
                                                            np.round(rmse,4), grid_model.best_params_))

In [16]:
# Lightgbm 학습 및 예측, 평가
from lightgbm import LGBMRegressor

Lgbm_reg = LGBMRegressor()
Lgbm_reg.fit(X_train, y_train)

get_rmse(Lgbm_reg)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 326
[LightGBM] [Info] Number of data points in the train set: 1070, number of used features: 11
[LightGBM] [Info] Start training from score 9.113489
LGBMRegressor 로그 변환된 RMSE: 17977.011


17977.0110494886

In [17]:
Lgbm_params1={ 'learning_rate':[0.01, 0.03, 0.05, 0.1],
             'num_leaves':[2, 4, 6, 8, 10],
              'subsample': [0.2, 0.4, 0.6, 0.8],
               'n_estimators':[50, 80, 100, 300],
             'colsample_bytree':[0.4, 0.6, 0.8],
              'reg_lambda': [2, 6, 10]
              }
print_best_params(Lgbm_reg, Lgbm_params1)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[LightGBM] [Info] Number of data points in the train set: 856, number of used features: 11
[LightGBM] [Info] Start training from score 9.136866
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000076 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 326
[LightGBM] [Info] Number of data points in the train set: 856, number of used features: 11
[LightGBM] [Info] Start training from score 9.107244
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000075 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 326
[LightGBM] [Info] Number of data points in the train set: 856, number of used features: 11
[LightGBM] [Info] Start training from score 9.090811
[LightGB

> 결과 : LGBMRegressor 5 CV 시 최적 평균 RMSE 값:0.3795, 최적 alpha:{'colsample_bytree': 0.8, 'learning_rate': 0.1, 'n_estimators': 50, 'num_leaves': 6, 'reg_lambda': 2, 'subsample': 0.2}


**결과로 학습하기**

In [18]:
from lightgbm import LGBMRegressor

lgbm_params={'n_esimators':[50]}
lgbm_reg = LGBMRegressor(n_estimators=50, learning_rate=0.1, num_leaves=6,
                         subsample=0.2, colsample_bytree=0.8, reg_lambda=2, n_jobs=-1)
print_best_params(lgbm_reg, lgbm_params)

[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.043266 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 326
[LightGBM] [Info] Number of data points in the train set: 856, number of used features: 11
[LightGBM] [Info] Start training from score 9.107244
[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Warning] Unknown parameter: n_esimators
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000088 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 326
[LightGBM] [Info] Number of data points in the train set: 856, number of

> LGBMRegressor 5 CV 시 최적 평균 RMSLE 값: 0.3795

In [20]:
lgbm_reg = None

lgbm_reg = LGBMRegressor(n_estimators=50, learning_rate=0.1, num_leaves=6,
                         subsample=0.2, colsample_bytree=0.8, reg_lambda=2, n_jobs=-1)
lgbm_reg.fit(X_train, y_train)

get_rmse(lgbm_reg)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000100 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 326
[LightGBM] [Info] Number of data points in the train set: 1070, number of used features: 11
[LightGBM] [Info] Start training from score 9.113489
LGBMRegressor 로그 변환된 RMSE: 17977.024


17977.024029908534

In [21]:
pred = lgbm_reg.predict(X_test)
pred = np.expm1(pred)
mse_pred = mean_squared_error(y_test, pred)
rmse_pred = np.sqrt(mse_pred)
rmse_pred

4475.290527964091